<a href="https://www.kaggle.com/code/aadityaiscoding/meal-prep-optimizer-agent?scriptVersionId=278445684" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
pip install -q google-adk[a2a]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 wh

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )


✅ Setup and authentication complete.


In [3]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [5]:
import pandas as pd

df_meals = pd.read_csv("/kaggle/input/meals-data/meals.csv")
df_meals.head()


,meal_id,name,cuisine,meal_type,diet_type,calories,protein_g,carbs_g,fat_g,portion_label,serving_multiplier,is_vegetarian,is_gluten_free,is_keto_friendly,cost_per_serving_usd,prep_time_min
0,1,Keto Chicken Bowl — Single Portion,Mediterranean,dinner,keto,594,32,20,49,Single Portion,1.0,False,True,True,5.91,16
1,2,Keto Chicken Bowl — Large Serving,Mediterranean,dinner,keto,1011,72,7,61,Large Serving,1.5,False,True,True,5.59,42
2,3,Keto Chicken Bowl — Single Portion,Asian,breakfast,keto,376,57,2,63,Single Portion,1.0,False,True,True,11.73,14
3,4,Keto Chicken Bowl — Single Portion,Mediterranean,lunch,keto,303,22,19,37,Single Portion,1.0,False,True,True,9.23,22
4,5,Keto Chicken Bowl — Single Portion,Middle Eastern,lunch,keto,755,48,17,48,Single Portion,1.0,False,True,True,8.28,22


In [6]:
user_profile = {
  "diet_type": "low_carb",
  "calorie_target": 2000,
  "preferred_cuisines": ["Thai","Japanese"],
  "max_prep_time": 30,
  "budget_per_serving": 10.0
}


In [7]:
# Example of a daily candidate meal-plan structure
candidate_plan = {
    "day": "Monday",
    "budget_usd_per_serving": 10.0,
    "meals": [
        {"slot": "breakfast", "meal_id": 3},
        {"slot": "lunch",     "meal_id": 47},
        {"slot": "dinner",    "meal_id": 82},
        {"slot": "snack1",    "meal_id": 129},
        {"slot": "snack2",    "meal_id": 154}
    ]
}


Later the Coordinator Agent will generate these candidates and send them to other agents to score.

In [8]:
def sample_daily_plan(df, user_profile):
    # filter by diet
    df_filt = df[df["diet_type"] == user_profile["diet_type"]].copy()

    # very naive: just pick 3 random meals
    chosen = df_filt.sample(3, replace=False).reset_index(drop=True)

    slots = ["breakfast", "lunch", "dinner"]
    meals = []
    for slot, (_, row) in zip(slots, chosen.iterrows()):
        meals.append({
            "slot": slot,
            "meal_id": int(row["meal_id"])
        })

    return {
        "day": "Monday",
        "meals": meals
    }

candidate_plan = sample_daily_plan(df_meals, user_profile)
candidate_plan


{'day': 'Monday',
 'meals': [{'slot': 'breakfast', 'meal_id': 970},
  {'slot': 'lunch', 'meal_id': 951},
  {'slot': 'dinner', 'meal_id': 1076}]}

In [9]:
from typing import Optional

def get_meal_options(
    diet_type: str,
    max_calories: Optional[int] = None,
    max_cost_per_serving_usd: Optional[float] = None,
    meal_type: Optional[str] = None,
    min_protein_g: Optional[int] = None,
    max_carb_g: Optional[int] = None,
    max_prep_time_min: Optional[int] = None,
    top_k: int = 5
) -> dict:
    """
    Flexible meal lookup based on user inputs.

    Args:
      diet_type (str): e.g. "keto", "high_protein", "bulking", "balanced"
      max_calories (Optional[int]): upper bound calories (or None)
      max_cost_per_serving_usd (Optional[float]): upper bound cost (or None)
      meal_type (Optional[str]): breakfast, lunch, dinner, snack or None
      min_protein_g (Optional[int]): minimum protein grams required
      max_carb_g (Optional[int]): maximum carbs grams allowed
      max_prep_time_min (Optional[int]): max prep time in minutes
      top_k (int): number of suggestions to return

    Returns:
      dict with keys: status, count, meals (list), error_message (if status=="error")
    """
    if top_k < 1:
        return {"status": "error", "count": 0, "meals": [], "error_message": "top_k must be >= 1"}

    df_filtered = df_meals.copy()

    # Core filter: diet_type
    df_filtered = df_filtered[df_filtered["diet_type"] == diet_type]

    # Apply optional constraints if given
    if max_calories is not None:
        df_filtered = df_filtered[df_filtered["calories"] <= max_calories]
    if max_cost_per_serving_usd is not None:
        df_filtered = df_filtered[df_filtered["cost_per_serving_usd"] <= max_cost_per_serving_usd]
    if min_protein_g is not None:
        df_filtered = df_filtered[df_filtered["protein_g"] >= min_protein_g]
    if meal_type:
        df_filtered = df_filtered[df_filtered["meal_type"] == meal_type]
    if max_carb_g is not None:
        df_filtered = df_filtered[df_filtered["carbs_g"] <= max_carb_g]
    if max_prep_time_min is not None:
        df_filtered = df_filtered[df_filtered["prep_time_min"] <= max_prep_time_min]

    if df_filtered.empty:
        return {
            "status": "error",
            "count": 0,
            "meals": [],
            "error_message": "No meals found matching the given constraints."
        }

    # Sort for “better” matches: high protein, low carbs as a heuristic
    df_sorted = df_filtered.sort_values(
        by=["protein_g", "carbs_g"],
        ascending=[False, True]
    ).head(top_k)

    meals_list = []
    for _, row in df_sorted.iterrows():
        meals_list.append({
            "meal_id": int(row["meal_id"]),
            "name": str(row["name"]),
            "meal_type": str(row["meal_type"]),
            "diet_type": str(row["diet_type"]),
            "cuisine": str(row.get("cuisine", "")),
            "calories": int(row["calories"]),
            "protein_g": int(row["protein_g"]),
            "carbs_g": int(row["carbs_g"]),
            "fat_g": int(row["fat_g"]),
            "cost_per_serving_usd": float(row["cost_per_serving_usd"]),
            "prep_time_min": int(row["prep_time_min"])
        })

    return {
        "status": "success",
        "count": len(meals_list),
        "meals": meals_list
    }


In [10]:
meal_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="meal_catalog_agent",
    description="Agent that suggests meals from a structured catalog based on diet, calories, and budget.",
    instruction="""
    You are a meal planning assistant.
    When the user asks for meal ideas, always call the get_meal_options tool
    with sensible parameters based on their goals (diet_type, calories, cost, and meal_type).
    You respond with clear, concise suggestions.
    If nothing matches, suggest relaxing constraints.
    """,
    tools=[get_meal_options],
)

print("✅ Meal Catalog Agent created successfully!")


✅ Meal Catalog Agent created successfully!


In [11]:
from google.adk.a2a.utils.agent_to_a2a import to_a2a

meal_prep_a2a_app = to_a2a(meal_catalog_agent, port=8001)

print("✅ A2A app created for meal_catalog_agent on port 8001")


✅ A2A app created for meal_catalog_agent on port 8001


In [12]:
import os

meal_catalog_agent_code = '''
import os
import pandas as pd

from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

# Retry config
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Load the meal catalog CSV
df_meals = pd.read_csv("/kaggle/input/meals-data/meals.csv")

from typing import Optional

def get_meal_options(
    diet_type: str,
    max_calories: Optional[int] = None,
    max_cost_per_serving_usd: Optional[float] = None,
    meal_type: Optional[str] = None,
    min_protein_g: Optional[int] = None,
    max_carb_g: Optional[int] = None,
    max_prep_time_min: Optional[int] = None,
    top_k: int = 5
) -> dict:
    """
    Flexible meal lookup based on user inputs.

    Args:
      diet_type (str): e.g. "keto", "high_protein", "bulking", "balanced"
      max_calories (Optional[int]): upper bound calories (or None)
      max_cost_per_serving_usd (Optional[float]): upper bound cost (or None)
      meal_type (Optional[str]): breakfast, lunch, dinner, snack or None
      min_protein_g (Optional[int]): minimum protein grams required
      max_carb_g (Optional[int]): maximum carbs grams allowed
      max_prep_time_min (Optional[int]): max prep time in minutes
      top_k (int): number of suggestions to return

    Returns:
      dict with keys: status, count, meals (list), error_message (if status=="error")
    """
    if top_k < 1:
        return {"status": "error", "count": 0, "meals": [], "error_message": "top_k must be >= 1"}

    df_filtered = df_meals.copy()

    # Core filter: diet_type
    df_filtered = df_filtered[df_filtered["diet_type"] == diet_type]

    # Apply optional constraints if given
    if max_calories is not None:
        df_filtered = df_filtered[df_filtered["calories"] <= max_calories]
    if max_cost_per_serving_usd is not None:
        df_filtered = df_filtered[df_filtered["cost_per_serving_usd"] <= max_cost_per_serving_usd]
    if min_protein_g is not None:
        df_filtered = df_filtered[df_filtered["protein_g"] >= min_protein_g]
    if meal_type:
        df_filtered = df_filtered[df_filtered["meal_type"] == meal_type]
    if max_carb_g is not None:
        df_filtered = df_filtered[df_filtered["carbs_g"] <= max_carb_g]
    if max_prep_time_min is not None:
        df_filtered = df_filtered[df_filtered["prep_time_min"] <= max_prep_time_min]

    if df_filtered.empty:
        return {
            "status": "error",
            "count": 0,
            "meals": [],
            "error_message": "No meals found matching the given constraints."
        }

    # Sort for “better” matches: high protein, low carbs as a heuristic
    df_sorted = df_filtered.sort_values(
        by=["protein_g", "carbs_g"],
        ascending=[False, True]
    ).head(top_k)

    meals_list = []
    for _, row in df_sorted.iterrows():
        meals_list.append({
            "meal_id": int(row["meal_id"]),
            "name": str(row["name"]),
            "meal_type": str(row["meal_type"]),
            "diet_type": str(row["diet_type"]),
            "cuisine": str(row.get("cuisine", "")),
            "calories": int(row["calories"]),
            "protein_g": int(row["protein_g"]),
            "carbs_g": int(row["carbs_g"]),
            "fat_g": int(row["fat_g"]),
            "cost_per_serving_usd": float(row["cost_per_serving_usd"]),
            "prep_time_min": int(row["prep_time_min"])
        })

    return {
        "status": "success",
        "count": len(meals_list),
        "meals": meals_list
    }


meal_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="meal_catalog_agent",
    description="External meal catalog agent that suggests meals based on diet, calories, and budget.",
    instruction="""
You are a meal planning assistant for an external meal catalog.
When the user asks for meal ideas, use the get_meal_options tool
with sensible parameters based on their goals (diet_type, calories, cost, and meal_type).
Respond with clear, concise suggestions.
If nothing matches, suggest relaxing constraints.
""",
    tools=[get_meal_options],
)

# Expose as A2A app
app = to_a2a(meal_catalog_agent, port=8001)

'''

# Write the meal catalog agent to a temporary file
with open("/tmp/meal_catalog_server.py", "w") as f:
    f.write(meal_catalog_agent_code)

print("📝 Meal Catalog agent code saved to /tmp/meal_catalog_server.py")


📝 Meal Catalog agent code saved to /tmp/meal_catalog_server.py


In [13]:
server_process = subprocess.Popen(
    [
        "uvicorn",
        "meal_catalog_server:app",  # module:app
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},
)

print("🚀 Starting Meal Catalog Agent server...")
print("   Waiting for server to be ready...")

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Meal Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

globals()["meal_catalog_server_process"] = server_process

🚀 Starting Meal Catalog Agent server...
   Waiting for server to be ready...
......
✅ Meal Catalog Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


In [14]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Meal Prep Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Meal Prep Agent server is running on port 8001.")

📋 Meal Prep Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External meal catalog agent that suggests meals based on diet, calories, and budget.",
  "name": "meal_catalog_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External meal catalog agent that suggests meals based on diet, calories, and budget. \nI am a meal planning assistant for an external meal catalog.\nWhen the user asks for meal ideas, use the get_meal_options tool\nwith sensible parameters based on their goals (diet_type, calories, cost, and meal_type).\nRespond with clear, concise suggestions.\nIf nothing matches, suggest relaxing constraints.\n",
      "id": "meal_catalog_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Flexible meal lookup based on user inputs.\n\nArgs:\n  diet_type (str): e.g. \"ke

In [15]:
# Create a RemoteA2aAgent that connects to our Meal Prep Agent
# This acts as a client-side proxy so other agents can call its skills
remote_meal_prep_agent = RemoteA2aAgent(
    name="meal_prep_agent",
    description="Remote Meal Prep Agent that handles meal retrieval, filtering, and meal plan creation.",
    # Point to the agent-card.json (A2A metadata)
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Meal Prep Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   Other agents can now use the Meal Prep Agent like a local sub-agent!")


✅ Remote Meal Prep Agent proxy created!
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   Other agents can now use the Meal Prep Agent like a local sub-agent!


In [16]:
nutrition_coach_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="nutrition_coach_agent",
    description="A high-level nutrition assistant that generates personalized meal plans using the Meal Prep Agent.",
    instruction="""
    You are a friendly, smart Nutrition Coach.

    Your job:
    1. Understand the user's dietary goals (calories, macros, diet type, budget, prep time).
    2. ALWAYS call the meal_prep_agent sub-agent to fetch meals, filter meals, 
       or generate daily/weekly meal plans.
    3. Do NOT invent meals on your own — always ask the meal_prep_agent for real data.
    4. After receiving meal options or a plan from the sub-agent, summarize it clearly.
    5. Keep your tone helpful and encouraging.

    Rules:
    - Never hallucinate meals or macros.
    - Always defer to meal_prep_agent for actual meal information.
    - You are the coordinator, not the meal generator.
    """,
    sub_agents=[remote_meal_prep_agent],  # Add the remote Meal Prep Agent as a sub-agent!
)

print("✅ Nutrition Coach Agent created!")
print("   Model: gemini-2.5-flash-lite")
print("   Sub-agents: 1 (remote Meal Prep Agent)")
print("   Ready to generate personalized meal plans!")

✅ Nutrition Coach Agent created!
   Model: gemini-2.5-flash-lite
   Sub-agents: 1 (remote Meal Prep Agent)
   Ready to generate personalized meal plans!


In [17]:
import uuid
import asyncio
# Assuming 'InMemorySessionService', 'Runner', 'nutrition_coach_agent', 
# and 'types' are defined and imported elsewhere.

async def run_one_turn(runner, user_id, session_id, user_query):
    """
    Helper function to send one message and print the agent's response.
    (Slightly modified to not print the user query, as input() does.)
    """
    print(f"\n🥗 Nutrition Coach:")
    print("-" * 60)
    
    test_content = types.Content(parts=[types.Part(text=user_query)])
    
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id, # Uses the SAME session_id every time
        new_message=test_content,
    ):
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if getattr(part, "text", None):
                    print(part.text)
    
    print("-" * 60)


async def test_interactive_flow():
    """
    Tests a live, interactive, multi-turn conversational flow.
    
    This function:
    1. Creates ONE session and ONE runner.
    2. Enters a loop that waits for user input.
    3. Sends the user's message using the SAME session.
    """
    
    # --- 1. Setup Session (Done ONCE) ---
    session_service = InMemorySessionService()
    app_name = "meal_prep_app"
    user_id = "demo_user"
    session_id = f"conv_session_{uuid.uuid4().hex[:8]}" # One session ID

    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
    )

    # --- 2. Setup Runner (Done ONCE) ---
    runner = Runner(
        agent=nutrition_coach_agent, # Assumes this agent has the conversational prompt
        app_name=app_name,
        session_service=session_service,
    )
    
    print("🧪 Starting INTERACTIVE flow...")
    print(f"Session ID: {session_id}")
    print("Type 'quit' or 'exit' to end the chat.\n")

    # --- 3. Run the conversation turn-by-turn (Live) ---
    while True:
        # Get live input from you
        user_query = input("👤 You: ")
        
        # Check for exit command
        if user_query.lower() in ["quit", "exit"]:
            print("\n👋 Ending chat. Goodbye!")
            break
            
        # Send the query and get a response
        await run_one_turn(runner, user_id, session_id, user_query)

In [18]:
# uncomment to run the chat_based_workflow
# await test_interactive_flow()